In [ ]:
import os
import pickle
import random
import numpy as np
import pandas as pd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
%matplotlib inline

## TCP mean performance

In [ ]:
tcp_loss_results = pickle.load(open("../results/tcp_loss_results.p", "rb"))
salt_pepper_loss_results = pickle.load(open("../results/salt_and_pepper_results.p", "rb"))
prefix = "../dataset/videos_tcp_loss"

In [ ]:
# compute single stream mean (without including drops)
num_frames_single = 0.0
mean_iou_single = 0.0
mean_c_single = 0.0
mean_prob_single = 0.0
for i in range(1, 7):
    k = os.path.join(prefix, str(i) + "_single")
    num_frames_single += len(tcp_loss_results[k]["iou"])
    mean_iou_single += np.sum(tcp_loss_results[k]["iou"])
    mean_c_single += np.sum(tcp_loss_results[k]["c"])
    mean_prob_single += np.sum(tcp_loss_results[k]["prob"])
    
mean_iou_single /= num_frames_single
mean_c_single /= num_frames_single
mean_prob_single /= num_frames_single

print mean_iou_single
print mean_c_single
print mean_prob_single

In [ ]:
# compute double stream mean
num_frames_double = 0.0
mean_iou_double = 0.0
mean_c_double = 0.0
mean_prob_double = 0.0
for i in range(1, 7):
    k = os.path.join(prefix, str(i) + "_dual")
    num_frames_double += len(tcp_loss_results[k]["iou"])
    mean_iou_double += np.sum(tcp_loss_results[k]["iou"])
    mean_c_double += np.sum(tcp_loss_results[k]["c"])
    mean_prob_double += np.sum(tcp_loss_results[k]["prob"])
    
mean_iou_double /= num_frames_double
mean_c_double /= num_frames_double
mean_prob_double /= num_frames_double

print mean_iou_double
print mean_c_double
print mean_prob_double

In [ ]:
# single stream mean (including drops)
print mean_iou_single * num_frames_single / num_frames_double
print mean_c_single * num_frames_single / num_frames_double
print mean_prob_single * num_frames_single / num_frames_double

## TCP visualization

In [ ]:
video_number = 3
num_frames = 300
k_single = os.path.join(prefix, str(video_number) + "_single")
k_double = os.path.join(prefix, str(video_number) + "_dual")

In [ ]:
kept_frames_single = pd.read_csv(os.path.join(prefix, "frames_kept_single.csv"), header=None)
kept_frames_double = pd.read_csv(os.path.join(prefix, "frames_kept_double.csv"), header=None)

In [ ]:
double_results = np.zeros(num_frames)
double_results[:] = np.nan

single_results = np.zeros(num_frames)
single_results[:] = np.nan

In [ ]:
ind = np.array(kept_frames_double.iloc[:, video_number - 1].dropna().tolist())
double_results[ind.astype(int)] = tcp_loss_results[k_double]["iou"]

ind = np.array(kept_frames_single.iloc[:, video_number - 1].dropna().tolist())
single_results[ind.astype(int)] = tcp_loss_results[k_single]["iou"]

no_loss_results = salt_pepper_loss_results["../dataset/" + str(video_number)][0.0]["iou"]

In [ ]:
plt.plot(no_loss_results, 'o', color="blue", alpha=0.8)
plt.plot(double_results, 'o', color="green", alpha=0.8)
plt.plot(single_results, 'o', color="red", alpha=0.8)
plt.xlabel("Video Frame")
plt.ylabel("Intersection over Union (IoU)")
plt.title("IoU by Frame")

blue_patch = mpatches.Patch(color="blue", label="No loss")
green_patch = mpatches.Patch(color="green", label="Dual TCP stream")
red_patch = mpatches.Patch(color="red", label="Single TCP stream")

plt.legend(handles=[blue_patch, green_patch, red_patch], loc="lower left")

plt.show()

## Salt and pepper mean performance

In [ ]:
salt_pepper_loss_results = pickle.load(open("../results/salt_and_pepper_results.p", "rb"))
salt_pepper_loss_df = pd.read_csv("../results/salt_and_pepper_df.csv")

In [ ]:
total_num_images = float(salt_pepper_loss_df["No. of images"].sum())
cols = salt_pepper_loss_df.columns.tolist()[2:]
for col in cols:
    val = (salt_pepper_loss_df[col] * salt_pepper_loss_df["No. of images"]).sum()
    val /= total_num_images
    print col + ": " + str(val)

## Salt and pepper visualization

In [ ]:
video_number = 1

plt.plot(salt_pepper_loss_results["../dataset/" + str(video_number)][0.0]["iou"], 'o', alpha=1.0, ms=5, color="blue")
plt.plot(salt_pepper_loss_results["../dataset/" + str(video_number)][0.01]["iou"], 'o', alpha=0.8, ms=3, color="green")
plt.plot(salt_pepper_loss_results["../dataset/" + str(video_number)][0.05]["iou"], 'o', alpha=0.8, ms=3, color="orange")
plt.plot(salt_pepper_loss_results["../dataset/" + str(video_number)][0.1]["iou"], 'o', alpha=0.8, ms=3, color="red")
plt.xlabel("Video Frame")
plt.ylabel("Intersection over Union (IoU)")
plt.title("IoU by Frame")

blue_patch = mpatches.Patch(color="blue", label="No loss")
green_patch = mpatches.Patch(color="green", label="1% loss")
orange_patch = mpatches.Patch(color="orange", label="5% loss")
red_patch = mpatches.Patch(color="red", label="10% loss")

plt.legend(handles=[blue_patch, green_patch, orange_patch, red_patch], loc="lower left")

plt.show()

In [ ]:
video_number = 1

plt.plot(salt_pepper_loss_results["../dataset/" + str(video_number)][0.0]["iou"], 'o', alpha=0.8, color="blue", ms=5)
plt.plot(salt_pepper_loss_results["../dataset/" + str(video_number)][0.0]["c"], 'o', alpha=0.8, color="green", ms=3)
plt.plot(salt_pepper_loss_results["../dataset/" + str(video_number)][0.0]["prob"], 'o', alpha=0.8, color="red", ms=3)
plt.xlabel("Video Frame")
plt.ylabel("Score")
plt.title("Scores by Frame (No Loss)")


blue_patch = mpatches.Patch(color="blue", label="IoU")
green_patch = mpatches.Patch(color="green", label="Confidence")
red_patch = mpatches.Patch(color="red", label="Probability")

plt.legend(handles=[blue_patch, green_patch, red_patch], loc="lower left")

plt.show()

## Drop loss mean performance

In [ ]:
drop_loss_df = pd.read_csv("../results/drop_loss_df.csv")

In [ ]:
print (drop_loss_df["IoU (p=0.1)"] * drop_loss_df["Image no. (p=0.1)"]).sum() / float(drop_loss_df["Image no. (p=0.1)"].sum())
print (drop_loss_df["IoU (p=0.2)"] * drop_loss_df["Image no. (p=0.2)"]).sum() / float(drop_loss_df["Image no. (p=0.2)"].sum())
print (drop_loss_df["IoU (p=0.3)"] * drop_loss_df["Image no. (p=0.3)"]).sum() / float(drop_loss_df["Image no. (p=0.3)"].sum())

In [ ]:
print (drop_loss_df["Conf. (p=0.1)"] * drop_loss_df["Image no. (p=0.1)"]).sum() / float(drop_loss_df["Image no. (p=0.1)"].sum())
print (drop_loss_df["Conf. (p=0.2)"] * drop_loss_df["Image no. (p=0.2)"]).sum() / float(drop_loss_df["Image no. (p=0.2)"].sum())
print (drop_loss_df["Conf. (p=0.3)"] * drop_loss_df["Image no. (p=0.3)"]).sum() / float(drop_loss_df["Image no. (p=0.3)"].sum())

In [ ]:
print (drop_loss_df["Prob. (p=0.1)"] * drop_loss_df["Image no. (p=0.1)"]).sum() / float(drop_loss_df["Image no. (p=0.1)"].sum())
print (drop_loss_df["Prob. (p=0.2)"] * drop_loss_df["Image no. (p=0.2)"]).sum() / float(drop_loss_df["Image no. (p=0.2)"].sum())
print (drop_loss_df["Prob. (p=0.3)"] * drop_loss_df["Image no. (p=0.3)"]).sum() / float(drop_loss_df["Image no. (p=0.3)"].sum())